# Welcome to PyPlanetSCA Demo using a Jupyter Notebook
## This demo is the package restructured to run in a Jupyter Notebook

## Start of the data gathering module
### This cell contains all necessary imports as well as allowing the user to switch the apiKey and output directory.

In [ ]:
import os
import requests
import geopandas as gpd
from requests.auth import HTTPBasicAuth
from shapely.geometry import shape
import numpy as np
import subprocess
import json
import pathlib
import pandas as pd
headers = {'Content-Type': 'application/json'}

############# SET THESE VARIABLES ######################
# redefined based on the filter you want to use
# from geojson_GIN import domain
# from geojson_STR import domain
# from geojson_DPO import domain
# from geojson_TUM import domain
# from geojson_869 import domain
# from geojson_DAN import domain
# from geojson_793 import domain
# from geojson_586 import domain
# from geojson_737 import domain
from geojson_551 import domain

domainID = '551'

# enter the Planet user API key - SWITCH
apiKey = ''
item_type = "PSScene"
asset_type = "ortho_analytic_4b_sr"
bundle_type = "analytic_sr_udm2"

# data download location - SWITCH
out_direc = ''
############# DON'T CHANGE VARIABLES BEYOND THIS ############

### Defines Functions for the data gathering module

In [ ]:
############ FUNCTIONS. DON'T CHANGE THESE!!! #########
def build_payload(item_ids, item_type, bundle_type, aoi_coordinates):
    payload = {
        "name": item_ids[0],
        "source_type": "scenes",
        "products": [
            {
                "item_ids": item_ids,
                "item_type": item_type,
                "product_bundle": bundle_type
            }
        ],
        "tools": [
            {
                "clip": {
                    "aoi": {
                        "type": "Polygon",
                        "coordinates": aoi_coordinates
                    }
                }
            }
        ]
    }
    return payload

def order_now(payload,apiKey):
    orders_url = 'https://api.planet.com/compute/ops/orders/v2'
    response = requests.post(orders_url, data=json.dumps(payload), auth=HTTPBasicAuth(apiKey, ''), headers=headers)
    print(response)

    if response.status_code==202:
        order_id =response.json()['id']
        url = f"https://api.planet.com/compute/ops/orders/v2/{order_id}"
        # feature_check = requests.get(url, auth=(PLANET_API_KEY, ""))
        feature_check = requests.get(url, auth=HTTPBasicAuth(apiKey, ''))
        if feature_check.status_code==200:
            print(f"Submitted a total of {len(feature_check.json()['products'][0]['item_ids'])} image ids: accepted a total of {len(feature_check.json()['products'][0]['item_ids'])} ids")
            print(f"Order URL: https://api.planet.com/compute/ops/orders/v2/{order_id}")
            return f"https://api.planet.com/compute/ops/orders/v2/{order_id}"
    else:
        print(f'Failed with Exception code : {response.status_code}')
        
def download_results(order_url,folder, overwrite=False):
    r = requests.get(order_url, auth=HTTPBasicAuth(apiKey, ''))
    try:
        if r.status_code ==200:
            response = r.json()
            results = response['_links']['results']
            results_urls = [r['location'] for r in results]
            results_names = [r['name'] for r in results]
            print('{} items to download'.format(len(results_urls)))

            for url, name in zip(results_urls, results_names):
                path = pathlib.Path(os.path.join(folder,name))

                if overwrite or not path.exists():
                    print('downloading {} to {}'.format(name, path))
                    r = requests.get(url, allow_redirects=True)
                    path.parent.mkdir(parents=True, exist_ok=True)
                    open(path, 'wb').write(r.content)
                else:
                    print('{} already exists, skipping {}'.format(path, name))
        else:
            print(f'Failed with response {r.status_code}')
    except:
        print('data not ready yet')
    r.close()
    # except Exception as e:
    #     print(e)
    #     print(order_url)
    #     raise Exception
    # r.close()

In [ ]:
## Ensure that the domain shape makes sense
domain_geometry = shape(domain['config'][0]['config'])
domain_geometry

In [ ]:
# Search API request object
search_endpoint_request = {
  "item_types": [item_type],
  "filter": domain
}
result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(apiKey, ''),
    json=search_endpoint_request)

In [ ]:
# View available data and prepare the list of planet IDs to download
geojson_data = result.json()
gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

# Add a new column to 'gdf' with the intersection area
gdf['intersection_area'] = gdf['geometry'].intersection(domain_geometry).area

# Calculate the percentage overlap
gdf['overlap_percentage'] = (gdf['intersection_area'] / domain_geometry.area) * 100

gdf

In [ ]:
id_list = [feature['id'] for idx, feature in enumerate(geojson_data['features']) if gdf['overlap_percentage'].iloc[idx] >= 99]
geom_list = [feature['geometry'] for idx, feature in enumerate(geojson_data['features']) if gdf['overlap_percentage'].iloc[idx] >= 99]
print(len(id_list))
print(sorted(id_list))

In [ ]:
### ONLY RUN THIS CELL IF YOU WANT TO CHECK ON ORDER STATUS

# # see the status of the requested tiles. Are they "active"?
# for IDD in id_list:
#     print(IDD)
#     command = 'curl -L -H "Authorization: api-key '+apiKey+'"'
#     sublink = " 'https://api.planet.com/data/v1/item-types/"+item_type+"/items/"+IDD+"/assets/' "
#     # sublink = " 'https://api.planet.com/data/v2/item-types/"+item_type+"/items/"+IDD+"/assets/' "
#     command = command+sublink+'| jq .'+asset_type+'.status'
#     status = subprocess.run(command, shell=True)
#     print(command)
#     # break

In [ ]:
# prepare and submit the orders
order_urls = pd.DataFrame(columns = ["index","ID_geom", "order_url"])

# loop through each order payload, and submit
for idx,IDD in enumerate(id_list):
    print(idx,IDD)
    
    payload = build_payload([IDD],item_type,bundle_type,domain['config'][0]['config']['coordinates'])
    order_url = order_now(payload,apiKey)
    
    order_urls.loc[idx, "index"] = idx        
    order_urls.loc[idx, "ID_geom"] = IDD
    order_urls.loc[idx, "order_url"] = order_url

In [ ]:
# check out the data, save to a csv if you want to come back later
print(order_urls)
order_urls.to_csv('urlSaver.csv', index = None)# save all URLs

In [ ]:
# download the orders once ready
# outputs of "data not ready yet" mean that the orders need more time to process before downloading
for url in order_urls.itertuples():
    print(url.index,url.order_url)
    print("start downloading data to".format(), out_direc + url.ID_geom)
    if url.order_url != None:
        try:
            nantest = ~np.isnan(url.order_url)
        except:
            download_results(url.order_url,folder = out_direc + url.ID_geom)
    # break

### Displays fp image to ensure that the data download has worked as necessary

In [ ]:
import rasterio
from rasterio.plot import show
#Replace with fp location
fp = ''
img = rasterio.open(fp)
show(img)

## Data prepration module
### Imports and defines SCA_Prediction

In [ ]:
pip install -U scikit-learn

In [ ]:
from rasterio import features
from rasterio.mask import mask
from rasterio.enums import MergeAlg
import matplotlib.pyplot as plt

import joblib
import time
import os
import string
import glob
from datetime import datetime
import sys
import json
import requests
from zipfile import ZipFile

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')
# rasterize the ROI for model training - USED
def vector_rasterize(dir_vector, dir_raster, dir_out, flag_output):
    vector = gpd.read_file(dir_vector)
    # Get list of geometries for all features in vector file
    geom = [shapes for shapes in vector.geometry]

    # Open example raster
    raster = rasterio.open(dir_raster)
    
    # reproject vector to raster
    vector = vector.to_crs(raster.crs)

    # create tuples of geometry, value pairs, where value is the attribute value you want to burn
    geom_value = ((geom,value) for geom, value in zip(vector.geometry, vector['label']))

    # Rasterize vector using the shape and transform of the raster
    rasterized = features.rasterize(geom_value,
                                    out_shape = raster.shape,
                                    transform = raster.transform,
                                    all_touched = True,
                                    fill = 9,   # background value
                                    merge_alg = MergeAlg.replace,
                                    dtype = np.float32)


    if flag_output:
        with rasterio.open(
                dir_out, "w",
                driver = "GTiff",
                transform = raster.transform,
                dtype = rasterio.float32,
                count = 1,
                width = raster.width,
                height = raster.height) as dst:
            dst.write(rasterized, indexes = 1)
    return rasterized

def run_sca_prediction(dir_raster, dir_out, nodata_flag, model):
    """
    This function predicts binary snow cover for planet satellite images using 
    the pre-trained random forest model 
    
    :param dir_raster: the directory or the file of planet images
    :param dir_out: the directory where output snow cover images will be stored
    :param nodata_flag: the value used to represent no data in the predicted snow cover image
    defult value is 9.
    model: the model used to predict snow cover
    
    """
    # if output directory not exist then creat the output directory
    if not os.path.exists(dir_out): os.mkdir(dir_out)
    
    # if dir_raster is a directory, then find all images with 'SR' flag, meaning surface reflectance data
    if os.path.isdir(dir_raster):
        file_list = glob.glob(dir_raster + './**/*SR*.tif', recursive = True)
    elif os.path.isfile(dir_raster):
        file_list = [dir_raster]

    print(file_list)
    for f in file_list:
        print('Start to predict:'.format(), os.path.basename(f))

        with rasterio.open(f, 'r') as ds:
            arr = ds.read()  # read all raster values
            if arr.shape[0] > 4: # if we have more than 4 bands
                arr = arr[:4,:,:] # use only the first four

        print("Image dimension:".format(), arr.shape)  # 
        X_img = pd.DataFrame(arr.reshape([4,-1]).T)
        X_img.columns = ['blue','green','red','nir']
        
        X_img = X_img/10000 # scale surface reflectance to 0-1
        
        X_img['nodata_flag'] = np.where(X_img['blue']==0, -1, 1) # wherever blue band is zero, set to nodata value of -1
        
        
        # run model prediction
        y_img = model.predict(X_img.iloc[:,0:4])
        
        out_img = pd.DataFrame()
        out_img['label'] = y_img
        out_img['nodata_flag'] = X_img['nodata_flag']
        out_img['label'] = np.where(out_img['nodata_flag'] == -1, nodata_flag, out_img['label']) # where we set to -1, set to new nodata_flag value
        # Reshape our classification map
        img_prediction = out_img['label'].to_numpy().reshape(arr[0,:, :].shape)

        
        file_out = dir_out + os.path.basename(f)[0:-4] + '_SCA.tif'
        print("Save SCA map to: ".format(),file_out)
        with rasterio.open(
                        file_out, "w",
                        driver = "GTiff",
                        transform = ds.transform,
                        dtype = rasterio.uint8,
                        count = 1,
                        crs = ds.crs,
                        width = ds.width,
                        height = ds.height,
                        nodata = nodata_flag) as dst:
                    dst.write(img_prediction, indexes = 1, masked=True)
                            


print("Done")

### Creates training and test data

In [ ]:
flag_rasterize = False

# dir_ROI = "./data/ROI/20180528_181110_add_UTM11.shp"
# dir_raster = "./data/planet/train/20180528_181110_1025_3B_AnalyticMS_SR_clip.tif"
# dir_ROIraster = './data/ROI/ROI_20180528_181110_add.tif'
# dir_samples_root = './data/samples/sample_'
# dir_samples = 'sample_174k.csv'

dir_ROI = ""
dir_raster = ""
dir_ROIraster = ''
dir_samples_root = ''
dir_samples = 'sample_174k.csv'

if flag_rasterize:
    flag_output = True
    # rasterize ROI
    ROI = vector_rasterize(dir_vector=dir_ROI, dir_raster=dir_raster, dir_out=dir_ROIraster, flag_output = flag_output)
    
    # save surface reflectance and lable to csv file
    N_scale = 10000.0
    img = rasterio.open(dir_raster)
    ROI = rasterio.open(dir_ROIraster)
    img_read = img.read()/N_scale
    df_img = pd.DataFrame(img_read.reshape([4,-1]).T)
    df_label = pd.DataFrame(ROI.read().reshape([1,-1]).T)

    df_train = pd.concat([df_img, df_label], axis = 1)
    df_train.columns = ['blue','green','red','nir','label']
    df_train['ndvi'] = (df_train['nir']-df_train['red'])/(df_train['nir']+df_train['red'])
    df_train = df_train[df_train.label !=9]
    df_train.label = np.where(df_train.label > 0, 1, 0)
    dir_samples = dir_samples_root + str(int(len(df_train.index)/1000)) + 'k.csv'
    df_train.to_csv(dir_samples, index = False)
else:
    df_train = pd.read_csv(dir_samples)

print("Done")

## Model Training Module
### Prints model results

In [ ]:
flag_train = False

# get data 
dir_model = 'random_forest_20240116_binary_174K.joblib'
dir_score = 'random_forest_20240116_binary_174K_scores.csv'
starttime = time.process_time()
if True:
    X = df_train[['blue', 'green','red','nir']]
    y = df_train['label']
    
    # pre-process ndvi value to -1.0 to 1.0; fill nan to finite value 
    # X[X['ndvi']< -1.0]['ndvi'] = -1.0
    # X[X['ndvi']> 1.0]['ndvi'] = 1.0
    # X[np.isfinite(X['ndvi']) == False]['ndvi'] = np.nan

    # define the model
    model = RandomForestClassifier(n_estimators=10, max_depth=10, max_features=4, random_state=1)#
    # evaluate the model
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
    n_accuracy = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    n_f1 = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    n_balanced_accuracy = cross_val_score(model, X, y, scoring='balanced_accuracy', cv=cv, n_jobs=-1, error_score='raise')
    
    # report performance
    plt.hist(n_f1)
    print('Repeat times:'.format(), len(n_f1))
    print('F1-score: %.5f (%.5f)' % (n_f1.mean(), n_f1.std()))
    print('Balanced Accuracy: %.5f (%.5f)' % (n_balanced_accuracy.mean(), n_balanced_accuracy.std()))
    print('Accuracy: %.5f (%.5f)' % (n_accuracy.mean(), n_accuracy.std()))

    # fit model with all observations
    model.fit(X,y)
    # save model 
    joblib.dump(model, dir_model)
    # save accuracy 
    scores = pd.DataFrame()
    scores["accuracy"] = n_accuracy
    scores['f1'] = n_f1
    scores['balanced_accuracy'] = n_balanced_accuracy
    scores.to_csv(dir_score, index = False)

    print('Total time used:'.format(), round(time.process_time() - starttime, 1))
print("Done")

## Prediction Evaluation module

In [ ]:
# run prediction for all images in a folder

#REPLACE
dir_raster = ''
dir_model = ""
dir_out = ''

#Custom code to run for single image
model = joblib.load(dir_model)
nodata_flag = 9
run_sca_prediction(dir_raster, dir_out, nodata_flag, model)
            
print("Done")

In [ ]:
#REPLACE
temp = rasterio.open("", 'r').read(masked = True)
plt.imshow(temp.squeeze())